# Improve our Agent "Second edition"

## - Install the Required Packages -

In [1]:
%pip install sentence-transformers -q

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install langchain -q
!pip install transformers -q
!pip install pinecone-client -q
!pip install faiss-cpu -q
!pip install SpeechRecognition -q
!pip install pyttsx3 -q
!pip install -q openai
!pip install -q -U langchain-community

In [3]:
%pip install -q openai==0.28
%pip install -q gTTS
%pip install -q langdetect

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
%pip install -q ipywebrtc
%pip install -q pvrecorder
%pip install -q sounddevice
%pip install -q gtts
%pip install -q openai


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\HUAWI\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


##  - set your OpenAI API key- 

In [5]:
# set OpenAI and chroma API key .env
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## - Load and Prepare JSON Data -


In [6]:
import json

with open("D:\Project\RimalAI_dataset_expanded.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []
metadatas = []
ids = []

for entry in data:
    # Concatenate relevant fields for embedding
    doc_text = f"{entry['name']} ({entry['type']}): {entry.get('description', '')} Vision 2030: {entry.get('vision2030', '')}"
    docs.append(doc_text)
    metadatas.append({"id": entry["id"], "type": entry["type"], "name": entry["name"]})
    ids.append(str(entry["id"]))


## - Create Embeddings and FAISS Vector Store -

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Use a sentence-transformers model for embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create the FAISS vector store
vectordb = FAISS.from_texts(
    texts=docs,
    embedding=embeddings,
    metadatas=metadatas
)

# Save the FAISS index for later use
vectordb.save_local("faiss_rimalai_db")
print("FAISS vector DB created and saved!")


C:\Users\HUAWI\AppData\Local\Temp\ipykernel_21488\3435374507.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


FAISS vector DB created and saved!


## - Query the Vector Database - 

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Custom model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


# Load the database
vectordb = FAISS.load_local("faiss_rimalai_db", embeddings, allow_dangerous_deserialization=True)

# query
query= "ancient Saudi cities"
results = vectordb.similarity_search(query, k=3)



# Print results
print("Results:")
for doc in results:
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("---")

Results:
Content: Al-Ula (landmark): Al-Ula is an ancient city located in northwestern Saudi Arabia, famous for its sandstone mountains, historic tombs, and rich Nabatean heritage. It has been a crossroads for ancient civilizations and a center of trade and culture. The city is home to significant archaeological sites like Mada'in Saleh, and its unique rock formations make it a prime location for tourists and historians alike. Vision 2030: Al-Ula is a centerpiece of Saudi Arabia's Vision 2030, aiming to transform the city into a world-class tourism destination while preserving its archaeological and cultural heritage. The city is also committed to sustainable tourism practices, ensuring that its natural beauty and historical value are maintained for future generations.
Metadata: {'id': 1, 'type': 'landmark', 'name': 'Al-Ula'}
---
Content: Role of Tribes in Unifying Saudi Arabia (history): The role of various tribes and key figures in the unification of Saudi Arabia, focusing on their c

## LangChain RetrievalQA ("gpt-4o") + Arabic Response

In [9]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-4 model
llm = ChatOpenAI(model_name="gpt-4o")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")
print("English Response:", response["result"])

# Arabic query
response = qa_chain("أخبرني عن مشاريع رؤية 2030 في السعودية.")
print("Arabic Response:", response["result"])

C:\Users\HUAWI\AppData\Local\Temp\ipykernel_21488\3646324446.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o")
C:\Users\HUAWI\AppData\Local\Temp\ipykernel_21488\3646324446.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")


English Response: Vision 2030 is Saudi Arabia's ambitious plan to diversify its economy and reduce its dependence on oil. Several key projects are part of this initiative, focusing on cultural, economic, and social development:

1. **Diriyah Gate Project**: This project aims to restore and develop Diriyah, the historic birthplace of the Saudi state, into a premier cultural and tourist destination. It focuses on showcasing the area's mud-brick architecture and its status as a UNESCO World Heritage site.

2. **Al-Ula Development**: Al-Ula is being transformed into a world-class tourism destination. Known for its rich Nabatean heritage and archaeological sites like Mada'in Saleh, the project emphasizes sustainable tourism to preserve its natural beauty and historical significance.

These projects reflect the broader goals of Vision 2030 to enhance tourism, preserve cultural heritage, and create new economic opportunities within the kingdom.
Arabic Response: تشمل رؤية 2030 في السعودية العد

## LangChain RetrievalQA (gpt-4) + Arabic Response

In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-4 model
llm = ChatOpenAI(model_name="gpt-4")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response_english = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")
print("English Response:", response_english["result"])

# Arabic query
response_arabic = qa_chain("أخبرني عن مشاريع رؤية 2030 في السعودية.")
print("Arabic Response:", response_arabic["result"])

English Response: Vision 2030 is a strategic framework for the future of Saudi Arabia, aiming to transform the country's economy and diversify it away from its heavy reliance on oil. It includes several key projects.

One such project involves the city of Al-Ula. This ancient city, known for its sandstone mountains and rich Nabatean heritage, is a centerpiece of Saudi Arabia's Vision 2030. The plan aims to transform Al-Ula into a world-class tourism destination, while preserving its archaeological and cultural heritage. Sustainable tourism practices are being implemented to ensure the city's natural beauty and historical value are maintained for future generations.

Another project under Vision 2030 is the Diriyah Gate project. Diriyah, known as the historic birthplace of the Saudi state, will be restored and developed as a premier cultural and tourist destination. This initiative is part of the effort to promote the country's history and culture.

Mohammed bin Salman, the Crown Prince

## LangChain RetrievalQA (gpt-3.5-turbo) + Arabic Response

In [12]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-3.5-turbo model
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response_english = qa_chain("Tell me about Vision 2030 projects in Saudi Arabia.")
print("English Response:", response_english["result"])

# Arabic query
response_arabic = qa_chain("أخبرني عن مشاريع رؤية 2030 في السعودية.")
print("Arabic Response:", response_arabic["result"])

English Response: Vision 2030 is a strategic initiative in Saudi Arabia aimed at diversifying the economy, reducing dependence on oil, and modernizing society. Two key projects under Vision 2030 are the Diriyah Gate project and the development of Al-Ula. 

The Diriyah Gate project focuses on revitalizing Diriyah, the historic birthplace of Saudi Arabia, into a premier cultural and tourist destination. This initiative involves restoring and developing the area's mud-brick architecture and historical sites to attract visitors and showcase Saudi Arabia's rich cultural heritage.

The development of Al-Ula aims to transform the ancient city into a world-class tourism destination while preserving its archaeological and cultural significance. This project emphasizes sustainable tourism practices to ensure that Al-Ula's natural beauty and historical value are maintained for future generations.
Arabic Response: رؤية 2030 في السعودية هي مبادرة شاملة تهدف إلى تحويل الاقتصاد السعودي وتنوي diversif

In [13]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize the GPT-4 model
llm = ChatOpenAI(model_name="gpt-4o")

# Create the QA chain with retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# English query
response_english = qa_chain("Tell me about Al-Ula and give me a video link.")
print("English Response:", response_english["result"])

# Arabic query
response_arabic = qa_chain("أخبرني عن العلا وأعطني رابط فيديو.")
print("Arabic Response:", response_arabic["result"])

English Response: Al-Ula is an ancient city located in northwestern Saudi Arabia, renowned for its stunning sandstone mountains, historic tombs, and the rich heritage of the Nabateans. It has historically been a crossroads for ancient civilizations and a hub of trade and culture. The city is home to significant archaeological sites such as Mada'in Saleh, and its unique rock formations make it a prime destination for tourists and historians. Al-Ula is also a key component of Saudi Arabia's Vision 2030, which aims to develop the city into a world-class tourism destination while preserving its archaeological and cultural heritage. The city is committed to sustainable tourism practices to ensure its natural beauty and historical significance are maintained for future generations.

Unfortunately, I can't provide a video link. However, you can search for Al-Ula videos on platforms like YouTube to visually explore this fascinating destination.
Arabic Response: العلا هي مدينة قديمة تقع في شمال

### Why did the model say "Unfortunately, I can't provide a video link"?

LLM Behavior:
The language model generates answers based on the text context it receives from your vector database. It does not automatically extract or highlight multimedia URLs unless the prompt or chain instructs it to.

Data in Vector Store:
Your dataset does contain video URLs for Al-Ula (e.g., "https://www.youtube.com/watch?v=u7cXaKYLyvs&t=315s"), but the vector store stores text embeddings of concatenated fields. The LLM sees the video URL as part of the text only if it was included in the chunk fed to it.

# fixed code with links to media provided in the prompt

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import re

# Set up LLM and RetrievalQA
llm = ChatOpenAI(model_name="gpt-4o")
prompt = PromptTemplate(
    template="""
You are a helpful assistant. Use the context below to answer the question.

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# Helper function to detect language
def detect_language(text):
    # Simple heuristic: checks if the text contains Arabic characters
    if re.search(r'[\u0600-\u06FF]', text):
        return 'arabic'
    else:
        return 'english'

# Main query function: Show only the top entity's media links
def query_with_auto_entity(query):
    # Detect the language of the query
    language = detect_language(query)

    docs = vectordb.similarity_search(query, k=5)
    entity_doc = docs[0] if docs else None
    if not entity_doc:
        return "No relevant entry found."
    
    # Get the response from the model (supports Arabic as well)
    response = qa_chain.invoke(query)["result"]
    
    # Retrieve media links from the entity document
    media = entity_doc.metadata.get("media", {})
    video_link = media.get("videos")[0] if media.get("videos") else None
    image_link = media.get("images")[0] if media.get("images") else None
    audio_link = media.get("audio")[0] if media.get("audio") else None
    
    # Prepare the response
    if language == 'arabic':
        # Arabic response
        result = response + "\n\nهذه الروابط ذات الصلة:\n"
        if video_link:
            result += f"- فيديو: [شاهد على يوتيوب]({video_link})\n"
        if image_link:
            result += f"- صورة: ![صورة]({image_link})\n"
        if audio_link:
            result += f"- صوت: [استمع هنا]({audio_link})\n"
        return f"Arabic Query Result:\n{result}"
    
    else:
        # English response
        result = response + "\n\nHere are the relevant media links:\n"
        if video_link:
            result += f"- Video: [Watch on YouTube]({video_link})\n"
        if image_link:
            result += f"- Image: ![Image]({image_link})\n"
        if audio_link:
            result += f"- Audio: [Listen here]({audio_link})\n"
        return f"English Query Result:\n{result}"

# Example usage with two queries: one in English, one in Arabic

# English query
query_english = "Tell me about Al-Ula"
answer_english = query_with_auto_entity(query_english)
print(answer_english)

# Arabic query
query_arabic = "أخبرني عن العلا"
answer_arabic = query_with_auto_entity(query_arabic)
print("\n")
print(answer_arabic)

English Query Result:
Al-Ula is an ancient city located in northwestern Saudi Arabia, renowned for its stunning sandstone mountains, historic tombs, and rich Nabatean heritage. It has historically been a crossroads for ancient civilizations and a hub of trade and culture. The city is home to significant archaeological sites, such as Mada'in Saleh, and its unique rock formations attract both tourists and historians. As part of Saudi Arabia's Vision 2030, Al-Ula is being developed into a world-class tourism destination. This initiative aims to enhance the city's cultural and historical appeal while preserving its archaeological sites and natural beauty. Al-Ula is also committed to sustainable tourism practices to ensure that its historical and environmental resources are protected for future generations.

Here are the relevant media links:



Arabic Query Result:
العلا هي مدينة قديمة تقع في شمال غرب المملكة العربية السعودية، وهي مشهورة بجبالها الرملية ومعالمها التاريخية وتراثها النبطي ال

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import langdetect

# 1. Set up the LLM and RetrievalQA chain
llm = ChatOpenAI(model_name="gpt-4o")

prompt = PromptTemplate(
    template="""
You are a helpful assistant. Use the context below to answer the question.

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# 2. Helper function to get the first media link by type
def get_first_media_link(media_dict, media_type):
    return media_dict.get(media_type, [None])[0]

# 3. Detect if the query is in Arabic
def detect_language(text):
    try:
        lang = langdetect.detect(text)
        return 'arabic' if lang == 'ar' else 'english'
    except:
        return 'english'

# 4. Main query function: Retrieves answer and media
def query_with_auto_entity(query):
    docs = vectordb.similarity_search(query, k=5)
    entity_doc = docs[0] if docs else None
    if not entity_doc:
        return "No relevant entry found."
    
    response = qa_chain.invoke(query)["result"].strip()

    media = entity_doc.metadata.get("media", {})
    video_link = get_first_media_link(media, "videos")
    image_link = get_first_media_link(media, "images")
    audio_link = get_first_media_link(media, "audio")

    language = detect_language(query)

    if language == 'arabic':
        result = f"{response}\n\nإليك الروابط ذات الصلة:\n"
        if video_link:
            result += f"- فيديو: [شاهد على يوتيوب]({video_link})\n"
        if image_link:
            result += f"- صورة: ![صورة]({image_link})\n"
        if audio_link:
            result += f"- صوت: [استمع هنا]({audio_link})\n"
    else:
        result = f"{response}\n\nHere are the relevant links:\n"
        if video_link:
            result += f"- Video: [Watch on YouTube]({video_link})\n"
        if image_link:
            result += f"- Image: ![Image]({image_link})\n"
        if audio_link:
            result += f"- Audio: [Listen here]({audio_link})\n"

    return result

# 5. Example usage
if __name__ == "__main__":
    queries = [
        "What is Saudi Coffee (Gahwa)?",
        "ما هي القهوة السعودية؟"
    ]
    for query in queries:
        print(f"\nQuery: {query}\n")
        print(query_with_auto_entity(query))
        print("="*80)



Query: What is Saudi Coffee (Gahwa)?

Saudi Coffee (Gahwa) is a traditional beverage in Saudi Arabia that symbolizes hospitality. It is often served with dates during social gatherings and is considered an important part of the country's cultural practices. As part of Saudi Arabia's Vision 2030, Saudi coffee is promoted as part of the nation's intangible cultural heritage to enhance cultural tourism.

Here are the relevant links:


Query: ما هي القهوة السعودية؟

القهوة السعودية هي نوع من القهوة التقليدية التي تُعدّ جزءًا من الثقافة والضيافة في المملكة العربية السعودية. تتميز القهوة السعودية عن أنواع القهوة الأخرى باستخدامها للبهارات مثل الهيل (الحبهان) والزعفران والقرنفل أحيانًا، مما يمنحها نكهة فريدة ومميزة. تُقدَّم القهوة السعودية عادةً في فناجين صغيرة وتُصاحبها التمر، وتعتبر جزءًا من الترحيب بالضيوف والتعبير عن الكرم والضيافة في المجتمع السعودي.

إليك الروابط ذات الصلة:



## 1. Speech-to-Text Using OpenAI Whisper API

In [13]:
import openai
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Function to transcribe audio using OpenAI's Whisper model
def transcribe_audio(file_path: str) -> str:
    # Open the audio file in binary mode
    with open(file_path, "rb") as audio_file:
        # Call the Whisper model to transcribe the audio
        transcript = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file
        )
    # Return the transcribed text
    return transcript['text']


In [14]:
# Usage English Audio
text = transcribe_audio("D:\Project\BedouinCulture.mp3")
print("Transcribed text:", text)

Transcribed text: Verbal poetry is the most popular art form in Bedouin culture. It was used not only as an art form, but also as a way to pass down stories through the generations, to convey information, and also to maintain social order in Bedouin society. Poets in Bedouin society were highly respected, and Bedouin poems included accounts of historical events, advice to children of the family, and accounts of battles. Traditionally, poems were recited around the campfire at night, and shared along with other stories. In ancient times, Bedouin couldn't read or write, so they relied on this type of verbal expression to pass on their traditions from one generation to the next.


In [15]:
# Usage Arabic Audio
text = transcribe_audio("D:\Project\ArabicAudio.m4a")
print("Transcribed text:", text)

Transcribed text: حكم سيوفك في رقاب العُذَّلي وإذا نزلت بدار ظلٍ فرحلي وإذا بليت بظالمٍ كن ظالمًا وإذا لقيت ذو الجهالة فجهلي وإذا الجبان نهاك يومك كريهةٍ خوفاً عليك من ازدحام الجحفلي فاعصي مقالته ولا تحفل بها واقدم إذا حق اللقى في الأول واختر لنفسك منزلاً تعلو به أو متكريماً تحت ظل القصطلي فالموت لا ينجيك من آفاته حصنٌ ولو شيدته بالجندلي موت الفتى في عزةٍ خير له من أن يبي تأسير طرفٍ أكحلي إن كنت في عدد العبيد فهمتي فوق الثريا والسماك الأعزلي أو أنكرت فرسان عبس النسبتي فسنان رمحي والحسام يقر لي وبذابلي ومهندي نلت العلا لا بالقرابة والعديد الأجزلي ورميت مهري في العجاج فخاضه والنار تقدح من شفار الأنصلي يا نازلين على الحما ودياره هل لا رأيتم في الديار تقلقلي طال عزكم وذلي في الهوى ومن العجاء بعزكم وتذللي لا تسقني ماء الحياة بذلة بل فاسقني بالعز كأس الحنظلي ماء الحياة بذلة كجهنم وجهنم بالعز أطيب منزلي لدعمنا قم بتسجيل الاعجاب بالفيديو والاشتراك في القناة مع مشاركة الفيديو


## 2. Text-to-Speech (TTS) Using ElevenLabs Voices

In [ ]:
import requests
from langdetect import detect
import os



def text_to_speech_elevenlabs(text, output_file, lang="en"):
    """
    Converts text to speech using ElevenLabs API and saves it as an audio file.
    
    Parameters:
    - text: The text to convert to speech.
    - output_file: The file name to save the output audio.
    - lang: The language of the text ('ar' for Arabic, 'en' for English).
    """
    try:
        # Select URL based on language (Arabic or English)
        if lang == "ar":
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_AR}"
        else:
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_EN}"

        data = {
            "text": text,
            "model_id": "eleven_monolingual_v1",  # Use "eleven_multilingual_v1" for multilingual support
            "voice_settings": {
                "stability": 0.75,
                "similarity_boost": 0.75
            }
        }

        headers = {
            "xi-api-key": ELEVEN_API_KEY,
            "Content-Type": "application/json"
        }

        # Send the request to the API
        response = requests.post(url, headers=headers, json=data)

        # Save the resulting audio file
        if response.status_code == 200:
            with open(output_file, "wb") as f:
                f.write(response.content)
            print(f"Audio saved successfully as {output_file}")
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"An error occurred during text-to-speech conversion: {e}")

def text_to_speech_elevenlabs_pipeline(text, output_file):
    """
    Detects the language of the text and converts it to speech accordingly.
    
    Parameters:
    - text: The text to be converted to speech.
    - output_file: The file name to save the output audio.
    """
    try:
        # Detect language ('en', 'ar', etc.)
        lang = detect(text)
    except Exception:
        lang = 'en'  # fallback to English if language detection fails

    print(f"Detected language: {lang}")
    
    # Convert text to speech using ElevenLabs
    text_to_speech_elevenlabs(text, output_file, lang)

def voice_query_pipeline(audio_file_path: str):
    """
    Handles a voice query: transcribes user speech, retrieves an answer, and converts it to speech.
    
    Parameters:
    - audio_file_path: Path to the input audio file containing the user's voice query.
    
    Returns:
    - output_audio: The path to the output audio file with the answer.
    """
    try:
        # 1. Transcribe user speech to text
        print("Transcribing user speech to text...")
        query_text = transcribe_audio(audio_file_path)
        print("User asked:", query_text)

        # 2. Get answer from LangChain RetrievalQA
        print("Fetching answer from LangChain RetrievalQA...")
        answer_text = query_with_auto_entity(query_text)
        print("Answer:", answer_text)

        # 3. Convert answer text to speech (using ElevenLabs for Arabic/English)
        output_audio = "answer.mp3"
        text_to_speech_elevenlabs_pipeline(answer_text, output_audio)

        # 4. Return or play output_audio as needed
        return output_audio

    except Exception as e:
        print(f"An error occurred during the voice query pipeline: {e}")
        return None

# Example usage
text_en = "Welcome to Al-Ula, a city of ancient wonders and modern vision."
text_ar = "مرحبًا بكم في العلا، مدينة العجائب القديمة والرؤية الحديثة."

# Convert text to speech
text_to_speech_elevenlabs_pipeline(text_en, "alula_en_elevenlabs.mp3")
text_to_speech_elevenlabs_pipeline(text_ar, "alula_ar_elevenlabs.mp3")

# Usage: Assuming the audio file path contains a valid user's question
audio_path = "user_question.wav"
output = voice_query_pipeline(audio_path)

if output:
    print(f"Audio output saved as {output}")
else:
    print("There was an error processing the query.")


Detected language: en
Audio saved successfully as alula_en_elevenlabs.mp3
Detected language: ar
Audio saved successfully as alula_ar_elevenlabs.mp3
Transcribing user speech to text...
User asked: I hope the light is here.
Fetching answer from LangChain RetrievalQA...
Answer: It seems like you are referring to the ambitious goals and initiatives under Saudi Arabia's Vision 2030. This vision, led by Mohammed bin Salman, aims to transform the country by reducing its reliance on oil, diversifying the economy, and enhancing various sectors such as tourism, technology, and cultural heritage. Projects like the Diriyah Gate and Neom are central to this vision, aiming to position Saudi Arabia as a leader in innovation, sustainability, and cultural heritage. If "I hope the light is here" refers to a hope for positive change and progress, Vision 2030 represents that hope by setting a new direction for the country.

Here are the relevant links:

Detected language: en
Audio saved successfully as an

In [17]:
def voice_query_pipeline(audio_file_path: str):
    # 1. Transcribe user speech to text
    query_text = transcribe_audio(audio_file_path)
    print("User asked:", query_text)

    # 2. Get answer from LangChain RetrievalQA
    answer_text = query_with_auto_entity(query_text)
    print("Answer:", answer_text)

    # 3. Convert answer text to speech using ElevenLabs (for Arabic/English)
    output_audio = "answer.mp3"
    text_to_speech_elevenlabs_pipeline(answer_text, output_audio)

    # 4. Return or play output_audio as needed
    return output_audio

In [19]:
# Usage
audio_path = "D:\Project\Q1.WAV"
voice_query_pipeline(audio_path)

User asked: Tell me about King Abdulaziz.
Answer: King Abdulaziz Al Saud, also known as Ibn Saud, was a pivotal figure in the history of Saudi Arabia. He played a crucial role in unifying the Arabian Peninsula, transforming it from a region of fragmented tribes and rival territories into a single, cohesive kingdom. His leadership was marked by a combination of military strategy, diplomacy, and a profound understanding of the cultural landscape. King Abdulaziz's efforts culminated in the establishment of the Kingdom of Saudi Arabia. He is recognized for his ability to unite various tribes and key figures under his leadership, facilitating the foundation of the modern state. His legacy is significant in Saudi history as he laid the groundwork for the nation's development and unity.

Here are the relevant links:

Detected language: en
Audio saved successfully as answer.mp3


'answer.mp3'

In [20]:
# Usage
audio_path = "D:\Project\Q2.WAV"
voice_query_pipeline(audio_path)

User asked: اخبرني عن العلم
Answer: يبدو أنك تسأل عن العلم في سياق رؤية 2030 أو تاريخ توحيد السعودية. إذا كنت تستفسر عن العلم في سياق رؤية 2030، فإنه يعكس جهود المملكة العربية السعودية في تعزيز التعليم والبحث العلمي كجزء من خطتها للتحول الوطني. تهدف رؤية 2030 إلى تحسين جودة التعليم وزيادة الاستثمار في البحوث العلمية والتطوير، وذلك لدعم الاقتصاد القائم على المعرفة.

أما إذا كنت تشير إلى العلم في سياق تاريخ توحيد المملكة، فإن التعليم قبل التوحيد كان محدودًا بسبب الظروف القاسية وقلة الموارد. ومع ذلك، بعد توحيد المملكة تحت قيادة الملك عبدالعزيز، بدأت الجهود لتحسين التعليم ونشر المعرفة كجزء من عملية بناء الدولة وتطويرها.

إليك الروابط ذات الصلة:

Detected language: ar
Audio saved successfully as answer.mp3


'answer.mp3'

## add recoding botton 

In [24]:
from pvrecorder import PvRecorder
import wave
import struct

def record_audio(output_file="user_question.wav", duration=5):
    recorder = PvRecorder(frame_length=512)
    recorder.start()
    frames = []
    
    # Record for 'duration' seconds
    for _ in range(int(duration * 16000 / 512)):  # 16000Hz / 512 samples-per-frame
        pcm_frames = recorder.read()
        # Convert 16-bit integer samples to bytes
        bytes_frames = [struct.pack('h', frame) for frame in pcm_frames]
        frames.extend(bytes_frames)
    
    recorder.stop()
    with wave.open(output_file, 'wb') as f:
        f.setnchannels(1)
        f.setsampwidth(2)  # 16-bit = 2 bytes
        f.setframerate(16000)
        f.writeframes(b''.join(frames))
    recorder.delete()

# Test recording
record_audio(duration=25)


In [ ]:
from pvrecorder import PvRecorder
import wave
import struct
import openai
from langdetect import detect
import requests
from IPython.display import Audio 

# ElevenLabs API key and voice IDs
ELEVEN_API_KEY = 
VOICE_ID_AR = 
VOICE_ID_EN = 

# Function to convert text to speech using ElevenLabs API
def text_to_speech_elevenlabs(text, output_file, lang="en"):
    try:
        if lang == "ar":
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_AR}"
        else:
            url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID_EN}"

        data = {
            "text": text,
            "model_id": "eleven_monolingual_v1",
            "voice_settings": {
                "stability": 0.75,
                "similarity_boost": 0.75
            }
        }

        headers = {
            "xi-api-key": ELEVEN_API_KEY,
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            with open(output_file, "wb") as f:
                f.write(response.content)
            print(f"Audio saved successfully as {output_file}")
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"An error occurred during text-to-speech conversion: {e}")

def record_audio(output_file="user_question.wav", duration=5):
    recorder = PvRecorder(frame_length=512)
    recorder.start()
    frames = []
    
    for _ in range(int(duration * 16000 / 512)):  
        pcm_frames = recorder.read()
        bytes_frames = [struct.pack('h', frame) for frame in pcm_frames]
        frames.extend(bytes_frames)
    
    recorder.stop()
    with wave.open(output_file, 'wb') as f:
        f.setnchannels(1)
        f.setsampwidth(2)
        f.setframerate(16000)
        f.writeframes(b''.join(frames))
    recorder.delete()

def voice_query_pipeline():
    # 1. Record audio
    record_audio() 
    
    # 2. Transcribe using Whisper
    audio_file = open("user_question.wav", "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    print("You asked:", transcript['text'])
    
    # 3. Get LangChain answer
    answer = query_with_auto_entity(transcript['text'])
    print("Answer:", answer)
    
    # 4. Convert answer to speech using ElevenLabs
    output_audio = "answer.mp3"
    lang = detect(answer)  # Detect the language of the answer
    text_to_speech_elevenlabs(answer, output_audio, lang)
    
    # 5. Return audio player
    return Audio(output_audio)

# Run the full pipeline
voice_query_pipeline()


You asked: Tell me about Al-Ula.
Answer: Al-Ula is an ancient city located in northwestern Saudi Arabia, renowned for its stunning sandstone mountains, historic tombs, and rich Nabatean heritage. It has served as a crossroads for ancient civilizations and has been a center of trade and culture throughout history. Al-Ula is home to significant archaeological sites, including Mada'in Saleh, which is famous for its well-preserved rock-cut tombs and monuments. The unique rock formations and historical significance of Al-Ula make it a prime destination for tourists and historians alike.

As part of Saudi Arabia's Vision 2030 initiative, Al-Ula is being developed into a world-class tourism destination, with a focus on preserving its archaeological and cultural heritage. The city is committed to sustainable tourism practices, which aim to maintain its natural beauty and historical value for future generations. This development is intended to boost tourism, create jobs, and diversify the econo

In [60]:
from gtts import gTTS
from IPython.display import Audio
import openai


def voice_query_pipeline():
    # 1. Record audio
    record_audio() 
    
    # 2. Transcribe using Whisper
    audio_file = open("user_question.wav", "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    print("You asked:", transcript['text'])
    
    # 3. Get LangChain answer
    answer = query_with_auto_entity(transcript['text'])
    print("Answer:", answer)
    
    # 4. Convert to speech
    tts = gTTS(answer)
    tts.save("answer.mp3")
    return Audio("answer.mp3")

# Run the full pipeline
voice_query_pipeline()



You asked: Tell me about Saudi culture.
Answer: Saudi culture is deeply rooted in its history and traditions, shaped by its geographical location and tribal heritage. Before the unification of Saudi Arabia in the early 20th century, the region consisted of fragmented tribal societies with a strong emphasis on clan loyalties and oral traditions. The harsh desert environment influenced the lifestyle and social structures of the people.

Key aspects of Saudi culture include:

1. **Tribal Heritage**: Tribal affiliations and loyalties have historically played a significant role in social and political life. These tribal ties were instrumental in the unification of the kingdom under King Abdulaziz.

2. **Coffee (Gahwa)**: Saudi coffee, known as Gahwa, is a traditional beverage that symbolizes hospitality. It is typically served with dates during social gatherings, highlighting the importance of hospitality and social interaction in Saudi society. Under Vision 2030, Saudi coffee is promoted a